In [1]:
import numpy as np
import pandas as pd
import os
import json
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


# Read from json files into csv files

In [2]:
# folder Path
path_dc = 'Dataset/2023-04-14-job-search/2023-04-14-job-search-location-DC/'
path_usa = 'Dataset/2023-04-14-job-search/2023-04-14-job-search-location-USA/'

# get all JSON file names as a list
# dc
json_files = [filename for filename in os.listdir(path_dc) if filename.endswith('.json')]

data_dc=pd.DataFrame()
for file_name in json_files:
    with open(os.path.join(path_dc, file_name)) as json_file:
        json_text = json.load(json_file)
        try:
            temp = pd.json_normalize(json_text, record_path=["jobs_results"], meta=["search_metadata", "search_parameters"])
        except:
            pass
        data_dc = pd.concat([data_dc, temp], axis=0)
print(data_dc.shape)
data_dc.to_csv("Dataset/csv_from_json/dc_results.csv", index=False)


# # usa
json_files = [filename for filename in os.listdir(path_usa) if filename.endswith('.json')]

data_usa=pd.DataFrame()
for file_name in json_files:
    with open(os.path.join(path_usa, file_name)) as json_file:
        json_text = json.load(json_file)
        try:
            temp = pd.json_normalize(json_text, record_path=["jobs_results"], meta=["search_metadata", "search_parameters"])
        except:
            pass
        data_usa = pd.concat([data_usa, temp], axis=0)
print(data_usa.shape)
data_usa.to_csv("Dataset/csv_from_json/usa_results.csv", index=False)

(372, 16)
(464, 15)


Check and delete duplicates for results of dc and usa respectively:

In [3]:
dc = pd.read_csv("./Dataset/csv_from_json/dc_results.csv")
usa = pd.read_csv("./Dataset/csv_from_json/usa_results.csv")
print(dc.duplicated().sum())
print(usa.duplicated().sum())

6
7


In [4]:
dc.drop_duplicates(ignore_index=True, inplace=True)
usa.drop_duplicates(ignore_index=True, inplace=True)
print(dc.shape)
print(usa.shape)

(366, 16)
(457, 15)


In [5]:
dc.to_csv("./Dataset/cleaned/dc_results.csv", index=False)
usa.to_csv("./Dataset/cleaned/usa_results.csv", index=False)

Combine all records from dc and usa:

In [6]:
all_res = pd.concat([dc, usa], ignore_index=True)
print(all_res.duplicated().sum())
all_res.drop_duplicates(ignore_index=True, inplace=True)
print(all_res.shape)
all_res.to_csv("./Dataset/cleaned/all_results.csv", index=False)

0
(823, 16)


In [6]:
print(dc.columns)
print(usa.columns)

Index(['title', 'company_name', 'location', 'via', 'description',
       'job_highlights', 'related_links', 'extensions', 'job_id',
       'detected_extensions.schedule_type', 'detected_extensions.salary',
       'detected_extensions.posted_at', 'search_metadata', 'search_parameters',
       'detected_extensions.work_from_home',
       'detected_extensions.commute_time'],
      dtype='object')
Index(['title', 'company_name', 'location', 'via', 'description',
       'job_highlights', 'related_links', 'extensions', 'job_id',
       'detected_extensions.posted_at', 'detected_extensions.schedule_type',
       'detected_extensions.salary', 'search_metadata', 'search_parameters',
       'detected_extensions.work_from_home'],
      dtype='object')


# Cleaning

In [110]:
data = pd.read_csv("./Dataset/csv_from_json/all_results.csv")
data.head(2)

,title,company_name,location,via,description,job_highlights,related_links,extensions,job_id,detected_extensions.schedule_type,detected_extensions.salary,detected_extensions.posted_at,search_metadata,search_parameters,detected_extensions.work_from_home,detected_extensions.commute_time
0,"Cloud Consultant, Big Data and Analytics, Goog...",Google,"Reston, VA",via Karkidi,Minimum qualifications:\n• Bachelor’s degree i...,"[{'title': 'Qualifications', 'items': ['Bachel...","[{'link': 'http://www.google.com/', 'text': 'g...","['120K–190K a year', 'Full-time']",eyJqb2JfdGl0bGUiOiJDbG91ZCBDb25zdWx0YW50LCBCaW...,Full-time,120K–190K a year,NaN,"{'id': '643dbb162fe3026efeedbf03', 'status': '...","{'q': 'big data and cloud computing', 'engine'...",NaN,NaN
1,Big Data Architect,Johnson Technology Systems Inc.,"Washington, DC",via Ladders,We are hiring for Big Data Architect candidate...,"[{'title': 'Qualifications', 'items': ['Minimu...","[{'link': 'http://www.jtsusa.com/', 'text': 'j...","['4 days ago', 'Full-time', 'No degree mention...",eyJqb2JfdGl0bGUiOiJCaWcgRGF0YSBBcmNoaXRlY3QiLC...,Full-time,NaN,4 days ago,"{'id': '643dbb162fe3026efeedbf03', 'status': '...","{'q': 'big data and cloud computing', 'engine'...",NaN,NaN


In [112]:
data["job_id"].duplicated().sum()

0

In [3]:
data.columns

Index(['title', 'company_name', 'location', 'via', 'description',
       'job_highlights', 'related_links', 'extensions', 'job_id',
       'detected_extensions.schedule_type', 'detected_extensions.salary',
       'detected_extensions.posted_at', 'search_metadata', 'search_parameters',
       'detected_extensions.work_from_home',
       'detected_extensions.commute_time'],
      dtype='object')

In [4]:
# rename some columns
data.rename(columns={"detected_extensions.schedule_type":"schedule_type", "detected_extensions.salary":"salary", 
                     "detected_extensions.posted_at":"posted_at", "detected_extensions.work_from_home":"work_from_home",
                     "detected_extensions.commute_time":"commute_time"}, inplace=True)

### Extract search query:

In [5]:
start = "{'q': "
end = ", 'engine'"
search_query = []
for i in data["search_parameters"]:
    q = re.search(f"{start}(.*){end}", i)
    try:
        q = q.group(1).strip().strip("'")
    except:
        q = ""
    search_query.append(q)
data["search_query"] = search_query

In [6]:
data.columns

Index(['title', 'company_name', 'location', 'via', 'description',
       'job_highlights', 'related_links', 'extensions', 'job_id',
       'schedule_type', 'salary', 'posted_at', 'search_metadata',
       'search_parameters', 'work_from_home', 'commute_time', 'search_query'],
      dtype='object')

In [7]:
data.drop(columns=["search_metadata","search_parameters","related_links","job_id","extensions","posted_at", "commute_time"], inplace=True)

In [8]:
data.head(2)

,title,company_name,location,via,description,job_highlights,schedule_type,salary,work_from_home,search_query
0,"Cloud Consultant, Big Data and Analytics, Goog...",Google,"Reston, VA",via Karkidi,Minimum qualifications:\n• Bachelor’s degree i...,"[{'title': 'Qualifications', 'items': ['Bachel...",Full-time,120K–190K a year,NaN,big data and cloud computing
1,Big Data Architect,Johnson Technology Systems Inc.,"Washington, DC",via Ladders,We are hiring for Big Data Architect candidate...,"[{'title': 'Qualifications', 'items': ['Minimu...",Full-time,NaN,NaN,big data and cloud computing


### clean column 'via'

In [9]:
platform = []
for i in data["via"]:
    i = i.replace("via ", "").strip()
    platform.append(i)
data["platform"] = platform

data.drop(columns=["via"], inplace=True)

In [10]:
for i in range(len(data["platform"])):
    if len(data.loc[i, "platform"].split("-"))>1:
        data.loc[i, "platform"] = data.loc[i, "platform"].split("-")[-1].strip(" ")

### clean column "job_highlights"

In [11]:
highlights=[]
for i in data["job_highlights"]:
    i = i.replace("'", "\"")
    highlights.append(i)

data["job_highlights"] = highlights
data.head(3)

,title,company_name,location,description,job_highlights,schedule_type,salary,work_from_home,search_query,platform
0,"Cloud Consultant, Big Data and Analytics, Goog...",Google,"Reston, VA",Minimum qualifications:\n• Bachelor’s degree i...,"[{""title"": ""Qualifications"", ""items"": [""Bachel...",Full-time,120K–190K a year,NaN,big data and cloud computing,Karkidi
1,Big Data Architect,Johnson Technology Systems Inc.,"Washington, DC",We are hiring for Big Data Architect candidate...,"[{""title"": ""Qualifications"", ""items"": [""Minimu...",Full-time,NaN,NaN,big data and cloud computing,Ladders
2,Data Architect,Applied Information Sciences,"Reston, VA","As a Data Architect, you will use cutting-edge...","[{""title"": ""Qualifications"", ""items"": [""Minimu...",Full-time,NaN,NaN,big data and cloud computing,Applied Information Sciences


In [12]:
job_highlights = pd.DataFrame()
for i in range(len(data["job_highlights"])):
    try:
        temp = pd.DataFrame(json.loads(f'{data["job_highlights"][i]}')).transpose().reset_index().loc[:,[0,1]]
        #set column names equal to values in row index position 0
        temp.columns = temp.iloc[0]
        #remove first row from DataFrame
        temp = temp.iloc[1:,[-2,-1]]
    except:
        temp = pd.DataFrame({"Qualifications":[""],"Responsibilities":[""],"Benefits":[""]})
    job_highlights = pd.concat([job_highlights, temp], axis=0)
job_highlights.reset_index(inplace=True)
job_highlights = job_highlights.iloc[:,1:]
job_highlights.head(3)


,Qualifications,Responsibilities,Benefits
0,"[Bachelor’s degree in Computer Science, Engine...","[As a Big Data and Analytics Cloud Consultant,...",NaN
1,[Minimum 8-12 years of experience in Data Mode...,[A Big Data Architect is responsible for desig...,NaN
2,[Minimum ten years of relational data backgrou...,[Work in a team with other smart AIS employees...,NaN


In [13]:
print(job_highlights.shape)

(823, 3)


In [14]:
data = pd.concat([data, job_highlights], axis=1)
print(data.columns)

Index(['title', 'company_name', 'location', 'description', 'job_highlights',
       'schedule_type', 'salary', 'work_from_home', 'search_query', 'platform',
       'Qualifications', 'Responsibilities', 'Benefits'],
      dtype='object')


In [15]:
data.head(2)

,title,company_name,location,description,job_highlights,schedule_type,salary,work_from_home,search_query,platform,Qualifications,Responsibilities,Benefits
0,"Cloud Consultant, Big Data and Analytics, Goog...",Google,"Reston, VA",Minimum qualifications:\n• Bachelor’s degree i...,"[{""title"": ""Qualifications"", ""items"": [""Bachel...",Full-time,120K–190K a year,NaN,big data and cloud computing,Karkidi,"[Bachelor’s degree in Computer Science, Engine...","[As a Big Data and Analytics Cloud Consultant,...",NaN
1,Big Data Architect,Johnson Technology Systems Inc.,"Washington, DC",We are hiring for Big Data Architect candidate...,"[{""title"": ""Qualifications"", ""items"": [""Minimu...",Full-time,NaN,NaN,big data and cloud computing,Ladders,[Minimum 8-12 years of experience in Data Mode...,[A Big Data Architect is responsible for desig...,NaN


In [16]:
data.isna().sum()

title                 0
company_name          0
location              0
description           0
job_highlights        0
schedule_type         1
salary              690
work_from_home      646
search_query          0
platform              0
Qualifications       12
Responsibilities     25
Benefits            393
dtype: int64

In [17]:
data.drop(columns=["job_highlights"], inplace=True)

In [18]:
data.isna().sum()

title                 0
company_name          0
location              0
description           0
schedule_type         1
salary              690
work_from_home      646
search_query          0
platform              0
Qualifications       12
Responsibilities     25
Benefits            393
dtype: int64

In [19]:
data.describe()

,title,company_name,location,description,schedule_type,salary,work_from_home,search_query,platform,Qualifications,Responsibilities,Benefits
count,823,823,823,823,822,133,177,823,823,811,798,430
unique,458,425,170,579,4,62,1,11,188,283,272,20
top,Data Analyst,Upwork,Anywhere,"First, The Data!\n• Over 1 million unique chat...",Full-time,20–28 an hour,True,big data and cloud computing,LinkedIn,,,
freq,21,56,177,6,719,32,177,90,66,393,393,393


### clean column "Benefits"

In [20]:
data.loc[data["Benefits"]=="","Benefits"]=None
data.loc[data["Qualifications"]=="","Qualifications"]=None
data.loc[data["Responsibilities"]=="","Responsibilities"]=None

In [21]:
data.isna().sum()

title                 0
company_name          0
location              0
description           0
schedule_type         1
salary              690
work_from_home      646
search_query          0
platform              0
Qualifications      405
Responsibilities    418
Benefits            786
dtype: int64

In [22]:
data.loc[data["Benefits"].isna()==False,"Benefits"].index

Int64Index([ 15,  21,  34,  46, 133, 185, 208, 209, 210, 228, 236, 249, 286,
            289, 302, 305, 314, 363, 366, 367, 372, 497, 564, 565, 578, 582,
            628, 631, 640, 672, 684, 728, 733, 736, 743, 747, 816],
           dtype='int64')

In [189]:
# keywords2 = ['\$', "per hour", "per year"]
# pattern2 = re.compile('|'.join(keywords2))

# sentences = re.split(r'(?<=\.)\s', " ".join(data["Benefits"][816]))

# res = str([s for s in sentences if pattern2.search(s)])[2:-2]
# res

### clean column 'salary'

Some information about salary/salary range are in column 'description' and "Benefits", so extract the salary in these two columns.

In [23]:
desc_salary = data["description"].str.contains("$",regex=True).values
benefit_salary = data["Benefits"].str.contains("$",regex=True).values
salary_isnull = data["salary"].isnull().values
salary_sentence = []

keywords1 = ['\$']
keywords2 = ['\$', "per hour", "per year"]
pattern1 = re.compile('|'.join(keywords1))
pattern2 = re.compile('|'.join(keywords2))

for i in range(len(desc_salary)):
    if (desc_salary[i]==True) and salary_isnull[i]==True:
        sentences = re.split(r'(?<=\.)\s', data["description"][i])
        matching_sentences = str([s for s in sentences if pattern1.search(s)])[2:-2]
        salary_sentence.append(matching_sentences)
    elif benefit_salary[i]==True and salary_isnull[i]==True:
        sentences = re.split(r'(?<=\.)\s', " ".join(data["Benefits"][i]))
        matching_sentences = str([s for s in sentences if pattern2.search(s)])[2:-2]
        salary_sentence.append(matching_sentences)
    else:
        salary_sentence.append(None)

data["salary_sentence"] = salary_sentence


In [24]:
data.loc[data["salary_sentence"]=="","salary_sentence"]=None
data["salary_sentence"].isna().sum()

545

In [25]:
salary_lower_bound = [None]*len(data["salary_sentence"])
salary_upper_bound = [None]*len(data["salary_sentence"])

# Regular expression to extract the substrings in the form of "$95,100.00-$218,700.00" or "$146,300 to $271,700"
pattern = r"\$([\d,]+(\.\d*)?)K*\s*(?:-|to|--|and)\s*\$([\d,]+(\.\d*)?)K*"

for i in range(len(data["salary_sentence"])):
    if data["salary_sentence"][i] :
        try:
            match = re.search(pattern, data["salary_sentence"][i])
            salary_lower_bound[i] = float(match.group(1).replace(",",""))
            salary_upper_bound[i] = float(match.group(3).replace(",",""))
        except:
            pass

data["salary_lower_bound"] = salary_lower_bound
data["salary_upper_bound"] = salary_upper_bound

In [26]:
data["salary_upper_bound"].isnull().sum()

651

In [27]:
data["salary_lower_bound"].unique()

array([        nan, 8.18000e+04, 1.31000e+05, 1.18000e+05, 3.00000e+01,
       1.05200e+05, 1.46300e+05, 9.24000e+04, 1.70000e+05, 5.84000e+04,
       7.80000e+04, 1.18300e+05, 9.10000e+04, 1.01200e+05, 9.33000e+04,
       7.31000e+04, 5.00000e+04, 1.37000e+05, 1.00000e+05, 1.10574e+05,
       9.91380e+04, 1.05900e+05, 2.00800e+05, 2.25000e+05, 8.60000e+04,
       8.64250e+04, 1.00900e+05, 1.50000e+05, 1.76000e+05, 1.40000e+05,
       1.30000e+05, 1.37300e+05, 1.96000e+05, 2.17350e+05, 0.00000e+00,
       1.12000e+05, 1.97400e+05, 6.59000e+04, 1.65000e+05, 1.42350e+05,
       1.51800e+05, 1.17545e+05, 4.00000e+01, 1.40000e+04, 1.50500e+05,
       1.16000e+05, 1.13500e+05, 5.09000e+03, 1.61000e+05, 1.28000e+05,
       1.60000e+05, 1.39000e+05, 1.09200e+05, 8.26000e+04, 1.02000e+05,
       1.05000e+05, 9.70000e+04, 8.50000e+04, 6.00000e+04, 5.81610e+04,
       8.90000e+04, 9.00000e+04, 7.71200e+04, 9.18960e+04, 9.95200e+04,
       6.60000e+04, 1.69000e+05, 1.20000e+02, 1.26750e+05, 1.175

In [28]:
data["salary_upper_bound"].unique()

array([          nan, 1.8600000e+05, 1.4000000e+05, 1.7700000e+05,
       3.5000000e+01, 1.6840000e+05, 2.7170000e+05, 2.0460000e+05,
       1.9000000e+05, 1.3300000e+05, 1.2000000e+05, 1.8200000e+05,
       1.8400000e+05, 1.9880000e+05, 2.1200000e+05, 1.6600000e+05,
       7.5000000e+04, 1.4500000e+05, 1.8797600e+05, 1.2737700e+05,
       2.5100000e+05, 2.4760000e+05, 1.3800000e+05, 1.5916500e+05,
       1.9330000e+05, 2.0000000e+05, 3.3350000e+05, 1.6000000e+05,
       2.4200000e+05, 2.5510000e+05, 2.8000000e+04, 3.4709000e+05,
       0.0000000e+00, 1.7900000e+05, 2.2530000e+05, 1.0410000e+05,
       2.2000000e+05, 2.1900000e+05, 2.1000000e+05, 1.5574400e+05,
       5.0000000e+01, 3.0000000e+05, 2.2550000e+05, 3.6000000e+05,
       2.0700000e+05, 1.0120000e+04, 2.7800000e+05, 1.5600000e+05,
       2.1300000e+05, 7.0000000e+05, 2.0280000e+05, 1.5340000e+05,
       1.4200000e+05, 1.2900000e+05, 7.1000000e+04, 1.0500000e+05,
       9.8873000e+04, 1.6900000e+05, 1.1000000e+05, 1.5844500e

Some information about salary/salary range has been recorded in the column 'salary', so salary lower/upper bound are extracted from this column. 

In [29]:
data["salary"].unique()

array(['120K–190K a year', nan, '20–28 an hour', '50K–100K a year',
       '10–30 an hour', '100K–200K a year', '150K–200K a year',
       '136K–146K a year', '60K–70K a year', '100K–500K a year',
       '100–120 an hour', '384,458 a year', '132,368 a year',
       '35–70 an hour', '30–60 an hour', '95K–115K a year',
       '93.8K–174K a year', '196K–335K a year', '53.3K–111K a year',
       '110K–159K a year', '130K–180K a year', '100K–150K a year',
       '35–50 an hour', '80–100 an hour', '10–80 an hour',
       '140K–160K a year', '80K–120K a year', '100K–140K a year',
       '130K–185K a year', '10–25 an hour', '15–20 an hour',
       '80K–140K a year', '140K–200K a year', '150K–250K a year',
       '75K–95K a year', '17.21 an hour', '60K–90K a year',
       '6,151.93–12,187.25 a month', '49,916–56,155 a year',
       '8,413–14,201 a month', '121K a year', '184K–275K a year',
       '98.9K–175K a year', '100K–125K a year', '130K–200K a year',
       '163K–245K a year', '125K–170K 

In [30]:
for i in range(len(data["salary"])):
    if salary_isnull[i]==False:
        if "a year" in str(data["salary"][i]):
            try:
                match = re.search(r"(\d+\W\d+)\D*–(\d+\W\d+)", data["salary"][i])  # like "49,916–56,155 a year"
                if match:
                    data.loc[i,"salary_lower_bound"] = float(match.group(1).replace(",","")) * 1000
                    data.loc[i,"salary_upper_bound"] = float(match.group(2).replace(",","")) * 1000
            except:
                pass

            try:
                match = re.search(r"(\d+\W*\d+)K–(\d+\W*\d+)K", data["salary"][i])  # like '120K–190K a year'
                if match:
                    data.loc[i,"salary_lower_bound"] = float(match.group(1)) * 1000
                    data.loc[i,"salary_upper_bound"] = float(match.group(2)) * 1000
            except:
                pass

            try:
                match = re.search(r"(\d+\W\d+)", data["salary"][i])  # like '132,368 a year'
                if match:
                    data.loc[i,"salary_lower_bound"] = None
                    data.loc[i,"salary_upper_bound"] = float(match.group().replace(",",""))
            except:
                pass
            
        elif "an hour" in str(data["salary"][i]):
            try:
                match = re.search(r"(\d+)–(\d+)", data["salary"][i])  # like "20–28 an hour"
                if match:
                    data.loc[i,"salary_lower_bound"] = float(match.group(1)) * 10400
                    data.loc[i,"salary_upper_bound"] = float(match.group(2)) * 10400
            except:
                pass

            try:
                match = re.search(r"(\d+\W*\d+)", data["salary"][i])  # like "82 an hour"
                if match:
                    data.loc[i,"salary_lower_bound"] = None
                    data.loc[i,"salary_upper_bound"] = float(match.group(2)) * 10400
            except:
                pass

        elif "a month" in str(data["salary"][i]):
            try:
                match = re.search(r"(\d+\W*\d+)\D*–(\d+\W*\d+)", data["salary"][i])  # like "8,413–14,201 a month"
                if match:
                    data.loc[i,"salary_lower_bound"] = float(match.group(1).replace(",","")) * 12
                    data.loc[i,"salary_upper_bound"] = float(match.group(2).replace(",","")) * 12
            except:
                pass

data["salary_lower_bound"].astype(float)
data["salary_upper_bound"].astype(float)

0      190000.0
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
818    100000.0
819    291200.0
820         NaN
821         NaN
822         NaN
Name: salary_upper_bound, Length: 823, dtype: float64

In [31]:
data.drop(columns=["salary","salary_sentence","description"], inplace=True)

In [32]:
data.isna().sum()

title                   0
company_name            0
location                0
schedule_type           1
work_from_home        646
search_query            0
platform                0
Qualifications        405
Responsibilities      418
Benefits              786
salary_lower_bound    591
salary_upper_bound    526
dtype: int64

In [33]:
data.to_csv("./Dataset/cleaned/temp.csv",index=False)


In [108]:
data = pd.read_csv("./Dataset/cleaned/temp.csv")
data.drop(columns="Benefits", inplace=True)
data.isna().sum()

title                   0
company_name            0
location                0
schedule_type           1
work_from_home        646
search_query            0
platform                0
Qualifications        405
Responsibilities      418
salary_lower_bound    586
salary_upper_bound    524
dtype: int64

#### fill in the missing values in columns 'salary_lower_bound' and 'salary_upper_bound'

In [109]:
# Filling nan values with mean
data['salary_lower_bound'] = data.groupby("search_query")['salary_lower_bound'].transform(lambda x: x.fillna(x.mean()))
data['salary_upper_bound'] = data.groupby("search_query")['salary_upper_bound'].transform(lambda x: x.fillna(x.mean()))

In [89]:
data.isna().sum()

title                   0
company_name            0
location                0
schedule_type           1
work_from_home        646
search_query            0
platform                0
Qualifications        405
Responsibilities      418
salary_lower_bound      6
salary_upper_bound      0
dtype: int64

### clean column "location"

In [90]:
cities = [None]*len(data["location"])
states = [None]*len(data["location"])

for i in range(len(data["location"])):
    data.loc[i,"location"] = data.loc[i,"location"].split("(")[0]
    if ('United States' not in data.loc[i,"location"]) and ("Anywhere" not in data.loc[i,"location"]) and (len(data.loc[i,"location"].split(","))>1):
        try:
            cities[i] = data.loc[i,"location"].split(",")[0].strip()
            states[i] = data.loc[i,"location"].split(",")[1].split(" ")[1].strip()
        except:
            pass
    elif ('United States' in data.loc[i,"location"]) or ("Anywhere" in data.loc[i,"location"]):
        cities[i] = "Anywhere"
        states[i] = "Anywhere"
    elif "other" in data.loc[i,"location"]:
        cities[i] = "Anywhere"
        states[i] = data.loc[i,"location"].split(" ")[1].strip()
    elif data.loc[i,"location"].strip(" ") == "Maryland":
        states[i] = "MD"
    elif data.loc[i,"location"].strip(" ") == "California":
        states[i] = "CA"
    elif data.loc[i,"location"].strip(" ") == "New York":
        states[i] = "NY"
    elif data.loc[i,"location"].strip(" ") == "Missouri":
        states[i] = "MO"
    elif data.loc[i,"location"].strip(" ") == "Illinois":
        states[i] = "IL"
    elif data.loc[i,"location"].strip(" ") == "Texas":
        states[i] = "TX"
    elif data.loc[i,"location"].strip(" ") == "Washington":
        states[i] = "WA"
    elif data.loc[i,"location"].strip(" ") == "Ohio":
        states[i] = "OH"

data["city"] = cities
data["state"] = states

data.loc[data["city"].isna()==True,"city"]="Anywhere"

In [91]:
data.drop(columns="location", inplace=True)

In [92]:
data["city"].unique()

array(['Reston', 'Washington', 'Chantilly', 'Laurel', 'Catonsville',
       'Annapolis', 'Hanover', 'Fort Meade', 'Columbia', 'Anywhere',
       'Annapolis Junction', 'Herndon', 'Baltimore', 'Wilmington',
       'Edison', 'Moline', 'Arlington', 'Manassas', 'Fairfax',
       'Rockville', 'New York', 'Seattle', 'San Francisco', 'Durham',
       'San Jose', 'Pittsburgh', 'Chicago', 'Alexandria', 'McLean',
       'Bethesda', 'Chevy Chase', 'Falls Church', 'Silver Spring',
       'Springfield', 'Beltsville', 'Germantown', 'Bowie', 'Takoma Park',
       'Wheaton-Glenmont', 'Fort Belvoir', 'Vienna', 'Sterling',
       'Suffolk', 'Bridgeport', 'Cupertino', 'Dallas',
       'South San Francisco', 'Honolulu', 'Livermore', 'Fremont',
       'Menlo Park', 'College Park', 'Kirkland', 'Culver City', 'Boston',
       'Cincinnati', 'Oak Ridge', 'Montreal', 'Brooklyn', 'Los Altos',
       'Columbus', 'Sunnyvale', 'Sacramento', 'Atlanta', 'Madrid',
       'Palo Alto', 'Los Angeles', 'Covington', 'San Di

In [93]:
data["state"].unique()

array(['VA', 'DC', 'MD', 'OH', 'Anywhere', 'DE', 'NJ', 'IL', 'NY', 'WA',
       'CA', 'NC', 'PA', 'WV', 'TX', 'HI', 'MA', 'TN', 'MO', 'GA', 'AL',
       'KY', 'SC', 'AZ', 'LA', 'UT', 'KS', 'FL', 'CO', 'NV', 'AR', 'NE',
       'OK', 'WI', 'MN', 'IN', 'MI'], dtype=object)

In [94]:
data.isna().sum()

title                   0
company_name            0
schedule_type           1
work_from_home        646
search_query            0
platform                0
Qualifications        405
Responsibilities      418
salary_lower_bound      6
salary_upper_bound      0
city                    0
state                   0
dtype: int64

### clean columns "Qualifications" and "Responsibilities"

In [95]:
print(data.loc[0,"Qualifications"])
print(type(data.loc[0,"Qualifications"]))

['Bachelor’s degree in Computer Science, Engineering, technical field or equivalent practical experience', '3 years of experience project managing and delivering technical solutions', 'Experience in systems design and architecting or communicating systems interactions, including data flows, interfaces, APIs, and methods', 'Experience with architecting, developing, or maintaining technical solutions in virtualized environments']
<class 'str'>


In [96]:
temp = ' '.join(x for x in eval(data.loc[0,"Qualifications"]))
print(temp)
print(type(temp))

Bachelor’s degree in Computer Science, Engineering, technical field or equivalent practical experience 3 years of experience project managing and delivering technical solutions Experience in systems design and architecting or communicating systems interactions, including data flows, interfaces, APIs, and methods Experience with architecting, developing, or maintaining technical solutions in virtualized environments
<class 'str'>


In [97]:
for i in range(data.shape[0]):
    if not data["Qualifications"].isna()[i]:
        data.loc[i, "Qualifications"] = ' '.join(x for x in eval(data.loc[i,"Qualifications"]))
    if not data["Responsibilities"].isna()[i]:
        data.loc[i, "Responsibilities"] = ' '.join(x for x in eval(data.loc[i, "Responsibilities"]))

In [98]:
# define stopwords
stopwords_en = stopwords.words('english')
# add=['rt','wa','u']
# for sp in add: stopwords_en.append(sp)

# Cleaning column 'Qualifications' row by row
cleaned_qualifications =[]

for each_row in data["Qualifications"]:
	try:
		tmp=''
		# Removing hashtags and mentions
		tmp = re.sub("^RT\s@[A-Za-z0-9_]+:","", each_row)
		tmp = re.sub("@[A-Za-z0-9_]+","", tmp)
		tmp = re.sub("#[A-Za-z0-9_]+","", tmp)

		# Removing '\n'
		tmp = re.sub("\n","", tmp)

		# Removing links
		tmp = re.sub(r"http\S+", "", tmp)
		tmp = re.sub(r"www.\S+", "", tmp)
	
		# only keep char in string.printable
		tmp = "".join(filter(lambda x: x in string.printable, tmp))

		# Lowercasing all the letters
		tmp = tmp.lower()

		#BREAK INTO CHUNKS (SENTANCES OR OTHERWISE)
		sentences = nltk.tokenize.sent_tokenize(tmp)  #SENTENCES

		# CLEAN AND LEMMATIZE
		keep='0123456789abcdefghijklmnopqrstuvwxy'

		new_qualifications=''
		for sentence in sentences:
			new_sentence=''

			# REBUILD LEMITIZED SENTENCE
			for word in sentence.split():
			
				# ONLY KEEP CHAR IN "keep"
				tmp2=''
				for char in word: 
					if(char in keep): 
						tmp2=tmp2+char
					else:
						tmp2=tmp2+' '
				word=tmp2

				#-----------------------
				# LEMMATIZE THE WORDS
				lemmatizer = WordNetLemmatizer()
				new_word = lemmatizer.lemmatize(word)
				#-----------------------

				# REMOVE WHITE SPACES
				new_word=new_word.replace(' ', '')

				# BUILD NEW SENTANCE BACK UP
				if(new_word not in stopwords_en):
					if(new_sentence==''):
						new_sentence=new_word
					else:
						new_sentence=new_sentence+' '+new_word

			# SAVE
			if(new_qualifications==''):
				new_qualifications=new_sentence
			else:
				new_qualifications=new_qualifications+' '+new_sentence

		cleaned_qualifications.append(new_qualifications)

	except:
		cleaned_qualifications.append(None)



In [99]:
# Cleaning reviews row by row
cleaned_responsibilities = []

for each_row in data["Responsibilities"]:
	try:
		tmp=''
		# Removing hashtags and mentions
		tmp = re.sub("^RT\s@[A-Za-z0-9_]+:","", each_row)
		# tmp = re.sub("@[A-Za-z0-9_]+","", tmp)
		# tmp = re.sub("#[A-Za-z0-9_]+","", tmp)

		# Removing '\n'
		tmp = re.sub("\n","", tmp)

		# Removing links
		tmp = re.sub(r"http\S+", "", tmp)
		tmp = re.sub(r"www.\S+", "", tmp)
	
		# only keep char in string.printable
		tmp = "".join(filter(lambda x: x in string.printable, tmp))

		# Lowercasing all the letters
		tmp = tmp.lower()

		#BREAK INTO CHUNKS (SENTANCES OR OTHERWISE)
		sentences = nltk.tokenize.sent_tokenize(tmp)  #SENTENCES

		# CLEAN AND LEMMATIZE
		keep='0123456789abcdefghijklmnopqrstuvwxy'

		new_responsibilities=''
		for sentence in sentences:
			new_sentence=''

			# REBUILD LEMITIZED SENTENCE
			for word in sentence.split():
			
				# ONLY KEEP CHAR IN "keep"
				tmp2=''
				for char in word: 
					if(char in keep): 
						tmp2=tmp2+char
					else:
						tmp2=tmp2+' '
				word=tmp2

				#-----------------------
				# LEMMATIZE THE WORDS
				lemmatizer = WordNetLemmatizer()
				new_word = lemmatizer.lemmatize(word)
				#-----------------------

				# REMOVE WHITE SPACES
				new_word=new_word.replace(' ', '')

				# BUILD NEW SENTANCE BACK UP
				if(new_word not in stopwords_en):
					if(new_sentence==''):
						new_sentence=new_word
					else:
						new_sentence=new_sentence+' '+new_word

			# SAVE
			if(new_responsibilities==''):
				new_responsibilities=new_sentence
			else:
				new_responsibilities=new_responsibilities+' '+new_sentence

		cleaned_responsibilities.append(new_responsibilities)
	except:
		cleaned_responsibilities.append(None)

data["Responsibilities"] = cleaned_responsibilities

In [100]:
data.head()

,title,company_name,schedule_type,work_from_home,search_query,platform,Qualifications,Responsibilities,salary_lower_bound,salary_upper_bound,city,state
0,"Cloud Consultant, Big Data and Analytics, Goog...",Google,Full-time,NaN,big data and cloud computing,Karkidi,"Bachelor’s degree in Computer Science, Enginee...",big data analytics cloud consultant work direc...,120000.000000,190000.000000,Reston,VA
1,Big Data Architect,Johnson Technology Systems Inc.,Full-time,NaN,big data and cloud computing,Ladders,Minimum 8-12 years of experience in Data Model...,big data architect responsible designing imple...,176890.555556,275471.086957,Washington,DC
2,Data Architect,Applied Information Sciences,Full-time,NaN,big data and cloud computing,Applied Information Sciences,Minimum ten years of relational data backgroun...,work team smart ai employee use cuttingedge te...,176890.555556,275471.086957,Reston,VA
3,Senior Cloud Solutions Architect Secret Clearance,inforeliance,Full-time,NaN,big data and cloud computing,WayUp,NaN,None,176890.555556,275471.086957,Washington,DC
4,Cloud Software Engineer,WayUp,Full-time,NaN,big data and cloud computing,JobLeads,NaN,None,176890.555556,275471.086957,Washington,DC


In [101]:
data.to_csv("./Dataset/cleaned/all.csv",index=False)